In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
# Load Data
data = pd.read_csv('/Users/robinfischer/Desktop/pyfin_assig_data.csv')


In [ ]:
# Calculate formation period returns and hold period returns
data['form_rets'] = data.groupby('permno')['price'].transform(pd.Series.pct_change, periods=6, fill_method=None)
data['hold_rets'] = data.groupby('permno')['price'].transform(pd.Series.pct_change, periods=1, fill_method=None).shift(-2)


In [ ]:
# Construct indicator for a 'good' stock
data['good_stock'] = 0
data.loc[~data['form_rets'].isnull(), 'good_stock'] = 1

In [ ]:
# Assign stocks into 10 equal groups based on formation period returns
data['port_mom'] = data.loc[data['good_stock'] == 1].groupby('date')['form_rets'].transform(pd.qcut, q=10, labels=range(1, 11)).astype(str)



In [ ]:
# Calculate the mean returns for each portfolio
port_df_mom = data.loc[~data['port_mom'].isnull()].groupby(['date', 'port_mom'])['hold_rets'].mean().reset_index()

In [ ]:
# Pivot the data for easier analysis
port_df_mom = port_df_mom.pivot(index='date', columns='port_mom', values='hold_rets').reset_index()



In [ ]:
# Calculate the relative strength of each portfolio
port_df_mom['relative_strength'] = port_df_mom.iloc[:, -1] - port_df_mom.iloc[:, 1]

In [ ]:
# Select the portfolio with the highest relative strength
top_portfolio = port_df_mom['relative_strength'].idxmax()

In [ ]:
# Vizualisation
px.line(port_df_mom, x='date', y='relative_strength', labels={'value': 'Cumulative returns'}, title='Relative Strength Strategy').show()

